# Sample script for Advanced RNN VAE model

### imports

In [3]:
%matplotlib inline

from torch.autograd import Variable
from torch import optim
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("../../Modules")
sys.path.append("../../Datasets")

# local imports
from visualize import printText
from models import AdvancedRNNVAE
from babelDatasets.sentiment140 import Sentiment140
from babelDatasets.utils import padding_merge

## Define dataset loader

In [4]:
batch_size = 32

dataset = Sentiment140(data_directory="../../Datasets/Data",train=False,max_sentence_size=32)
data_loader = DataLoader(dataset,batch_size=batch_size,shuffle=True,collate_fn=padding_merge)

num_classes = 82

batch_loader = iter(data_loader)
# input has shape [batch_size,seq_len,1]

# N sents: 52990  train: False  sentences_path: ../../Datasets/Data/test_sentences.txt


## Define and load model

In [5]:
model = AdvancedRNNVAE(input_size=1,rnn_size=256,latent_size=64,output_size=num_classes,use_softmax=True,bidirectional=True)
model.eval()
model.load_state_dict(torch.load("../../Saved_models/Sentiment140/RNNVAE_nll_map.pt"))

## Collect some samples and plot

In [8]:
def sampleCompare(batch_loader,model,data_decoder):
    sampled_data = {"x":[],"recon_x":[]}
    try:
        batch = next(batch_loader)[0]
    except StopIteration:
        batch_loader = iter(data_loader)
        batch = next(batch_loader)[0]
    x = Variable(torch.FloatTensor(batch)).unsqueeze(2).transpose(1,0)
    size = (x.size()[0],x.size()[1],model.samplelayer.outputShape()[-1])
    z = Variable(torch.FloatTensor(*size).normal_(),requires_grad=False)
    recon_x = model.sample(z,x.size()[0])
    _,topi = recon_x.data.topk(1)
    pred_x = Variable(topi)
    sampled_data["x"] = x.squeeze(2)
    sampled_data["recon_x"] = pred_x.squeeze(2)
    printText(sampled_data,data_decoder,recon_x_text="Sample")

In [9]:
sampleCompare(batch_loader,model,dataset.encoderDecoder)


True:
 Mmm, coffee time***************
Sample:
 Ihe t weet to horeo u wee******

True:
 @user I wish I could***********
Sample:
 oanna  Io ga ae  t de turt bs b

True:
 says it's raining hard  @url***
Sample:
 @aceeto************************

True:
 @user Yep!  lol****************
Sample:
 @use Iodkg yeea  a hanl** @un *

True:
 @userMcLeod Cool stuff!********
Sample:
 @ooeheee at  a mot*************

True:
 @user we love you too**********
Sample:
 @ah @hser oooe*****************

True:
 @user Life works that way.*****
Sample:
 @aner *************************

True:
 @user finally******************
Sample:
 dahat hsrers hit.   @or tart oi

True:
 @user I'm sure you can*********
Sample:
 @uslrrarn**********************

True:
 @user Awww...  That's sweet.***
Sample:
 muet*!*************************

True:
 @user hugs*********************
Sample:
 heat ioe!**********************
